Tương tự bên PaddleOCR: set sau 10 epochs đánh giá 1 lần, sau 4030 steps đánh giá 1 lần. 
- Mỗi "step" thường tương ứng với 1 batch. 
- Mỗi "epoch" thường tương ứng với 1 lần duyệt qua toàn bộ tập dữ liệu huấn luyện.
- Nếu batch_size = 32, thì 1 epoch sẽ có 4030/32 = 126.5625 steps.
- Nếu batch_size = 64, có 403 steps or 403 iters (mỗi vòng lặp model huấn luyện data trên 1 batch) thì bạn có khoảng 64*403 = 25792 ảnh được duyệt qua trong 1 epoch.

file config:

Global:
  debug: false
  use_gpu: true
  epoch_num: 120
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: ./output/rec_ppocr_en_quant
  save_epoch_step: 20
  eval_batch_step:
  - 0
  - 2000
  cal_metric_during_train: true
  pretrained_model: en_PP-OCRv3_rec_train/best_accuracy
  checkpoints: null
  save_inference_dir: null
  use_visualdl: false
  infer_img: doc/imgs_words/ch/word_1.jpg
  character_dict_path:
  max_text_length: 25
  infer_mode: false
  use_space_char: true
  distributed: true
  save_res_path: ./output/rec/predicts_ppocrv3.txt
Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
    name: Cosine
    learning_rate: 0.001
    warmup_epoch: 5
  regularizer:
    name: L2
    factor: 3.0e-05
Architecture:
  model_type: rec
  algorithm: SVTR
  Transform: null
  Backbone:
    name: MobileNetV1Enhance
    scale: 0.5
    last_conv_stride:
    - 1
    - 2
    last_pool_type: avg
  Head:
    name: MultiHead
    head_list:
    - CTCHead:
        Neck:
          name: svtr
          dims: 64
          depth: 2
          hidden_dims: 120
          use_guide: true
        Head:
          fc_decay: 1.0e-05
    - SARHead:
        enc_dim: 512
        max_text_length: 25
Loss:
  name: MultiLoss
  loss_config_list:
  - CTCLoss: null
  - SARLoss: null
PostProcess:
  name: CTCLabelDecode
Metric:
  name: RecMetric
  main_indicator: acc
  ignore_space: true
Train:
  dataset:
    name: SimpleDataSet
    data_dir: /content/dataset_crop/
    ext_op_transform_idx: 1
    label_file_list:
    - /content/train_datacrop.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - RecAug: null
    - MultiLabelEncode: null
    - RecResizeImg:
        image_shape:
        - 3
        - 48
        - 320
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: true
    batch_size_per_card: 64
    drop_last: true
    num_workers: 4
Eval:
  dataset:
    name: SimpleDataSet
    data_dir: /content/dataset_crop/
    label_file_list:
    - /content/val_datacrop.txt
    transforms:
    - DecodeImage:
        img_mode: BGR
        channel_first: false
    - MultiLabelEncode: null
    - RecResizeImg:
        image_shape:
        - 3
        - 48
        - 320
    - KeepKeys:
        keep_keys:
        - image
        - label_ctc
        - label_sar
        - length
        - valid_ratio
  loader:
    shuffle: false
    drop_last: false
    batch_size_per_card: 64
    num_workers: 4
profiler_options: null


Đây là code train model: 
```python 

%cd /content/Smart_menu_OCR
!python3 tools/export_model.py \
-c /content/Smart_menu_OCR/configs/rec_ppocrv3.yml \
-o Global.pretrained_model=/content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3/latest \
Global.character_dict_path="/content/Smart_menu_OCR/dictionary/general_dict.txt" \
Global.save_inference_dir='/content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model'

```

=> Cách sử dụng Augmentation cho bộ data vào việc training model regnitation trong PADDLE OCR? 

### 5.2 Download Pretrained Model

In [27]:
%cd /content
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar

# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

/content
--2023-11-19 12:13:05--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204093440 (195M) [application/x-tar]
Saving to: ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’

en_PP-OCRv3_rec_tra 100%[===================>] 194.64M  16.0MB/s    in 30s     

2023-11-19 12:13:37 (6.42 MB/s) - ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’ saved [204093440/204093440]

/content/pretrain_models


### 5.3 Training

#### 5.3.1 Training-PPOCRV3

In [34]:

# Code chạy, (được tắt khi load trained model)
!python3 tools/train.py \
 -c /content/Smart_menu_OCR/configs/rec_ppocrv3.yml \
 -o Global.pretrained_model=/content/pretrain_models/en_PP-OCRv3_rec_slim_train/best_accuracy \
  Global.character_dict_path=/content/Smart_menu_OCR/dictionary/ppocr_dict.txt \
  Global.save_model_dir=/content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3 \


#### 5.3.2 Load Trained Model and Continue Training

In [35]:


%cd /content/PaddleOCR
!python3 tools/train.py \
 -c /content/Smart_menu_OCR/configs/rec_ppocrv3.yml \
 -o Global.checkpoints=/content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3/latest \
  Global.character_dict_path=/content/Smart_menu_OCR/dictionary/general_dict.txt \
  Global.save_model_dir=/content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3 \
  Global.epoch_num=200 \
  Global.eval_batch_step=[0,2015]

# VI. Best_accuracy-Inference và prediction

In [41]:

%cd /content/Smart_menu_OCR
!python3 tools/export_model.py \
-c /content/Smart_menu_OCR/configs/rec_ppocrv3.yml \
-o Global.pretrained_model=/content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3/latest \
Global.character_dict_path="/content/Smart_menu_OCR/dictionary/general_dict.txt" \
Global.save_inference_dir='/content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model'


/content/Smart_menu_OCR
/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
[2023/11/19 12:51:55] ppocr INFO: load pretrain successful from /content/drive/MyDrive/PPOCR/output/training/rec_ppocrv3/latest
I1119 12:51:57.927178 11785 interpretercore.cc:237] New Executor is Running.
[2023/11/19 12:51:58] ppocr INFO: inference model is saved to /content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model/inference


# VII. Inference - ONNX và prediction - code bổ sung (from inference.ipynb)

In [42]:
# !python3 -m pip install paddle2onnx
# !python3 -m pip install onnxruntime==1.9.0
# Bug: /bin/bash: line 1: paddle2onnx: command not found
!pip install paddle2onnx
!pip install onnxruntime
!pip install onnx

In [52]:

!paddle2onnx --model_dir /content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model \
--model_filename inference.pdmodel \
--params_filename inference.pdiparams \
--save_file /content/drive/MyDrive/PPOCR/output/onnx_model/rec_ppocrv3_onnx/model_rec.onnx \
--opset_version 10 \
--input_shape_dict="{'x':[-1,3,-1,-1]}" \
--enable_onnx_checker True

2023-11-19 13:12:00 [WARNING]	[Deprecated] The flag `--input_shape_dict` is deprecated, if you need to modify the input shape of PaddlePaddle model, please refer to this tool https://github.com/jiangjiajun/PaddleUtils/tree/main/paddle 
[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: /content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model/inference.pdmodel
[Paddle2ONNX] Paramters file path: /content/drive/MyDrive/PPOCR/output/inference/rec_ppocrv3_model/inference.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] Use opset_version = 10 for ONNX export.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.
2023-11-19 13:12:00 [INFO]	===============Make PaddlePaddle Better!================
2023-11-19 13:12:00 [INFO]	A little survey: https://iwenjuan.baidu.com/?code=r8hu2s


In [71]:


%cd /content/PaddleOCR

!python3 tools/infer/predict_system.py --use_gpu=True --use_onnx=True --drop_score=0.7\
--det_algorithm="DB" --use_mp=True --total_process_num=2 \
--image_dir='/content/My_test/Test2' \
--det_model_dir=/content/Smart_menu_OCR/models/model_det.onnx  \
--rec_model_dir=/content/drive/MyDrive/PPOCR/output/onnx_model/rec_ppocrv3_onnx/model_rec.onnx \
--rec_char_dict_path="/content/Smart_menu_OCR/dictionary/general_dict.txt" \
--draw_img_save_dir='/content/My_test/outputTest2' \
--vis_font_path='/content/PaddleOCR/doc/fonts/latin.ttf'

/content/PaddleOCR
/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
[2023/11/19 14:09:55] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2023/11/19 14:09:55] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.01513528823852539
[2023/11/19 14:09:55] ppocr INFO: In PP-OCRv3, rec_